In [ ]:
import os
import urllib.request
import datetime
import pandas as pd
import glob
from colorama import Fore, Style

In [ ]:
def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"{Fore.RED}Directory '{path}' created.")
    else:
        print(f"Directory '{path}' already exists.")

In [ ]:
def retrieve_vhi_data(region_code, year_start=1981, year_end=2024):
    base_url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php"
    query = f"?country=UKR&provinceID={region_code}&year1={year_start}&year2={year_end}&type=Mean"
    response = urllib.request.urlopen(base_url + query)
    
    if response.status == 200:
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')
        file_path = f"data/vhi_{region_code}_{timestamp}.csv"
        with open(file_path, 'wb') as file:
            file.write(response.read())
        print(f"{Fore.GREEN}Data for region {region_code} saved to {file_path}.")
    else:
        print(f"{Fore.RED}Failed to download data for region {region_code}.")

In [ ]:
def download_all_regions(start_region=1, end_region=27):
    for region in range(start_region, end_region + 1):
        retrieve_vhi_data(region)

ensure_dir('data')
download_all_regions()

In [ ]:
region_names_ukr = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька", 5: "Житомирська", 
    6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська", 9: "Київська", 10: "Кіровоградська", 
    11: "Луганська", 12: "Львівська", 13: "Миколаївська", 14: "Одеська", 15: "Полтавська", 
    16: "Рівненська", 17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська", 
    21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська", 25: "Республіка Крим"
}

In [ ]:
region_mappings = {
    1: 24, 2: 25, 3: 5, 4: 4, 5: 3, 6: 4, 7: 8, 8: 19, 9: 9, 10: 10, 
    11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 20, 
    20: 20, 21: 21, 22: 22, 23: 23, 24: 1, 25: 25, 26: 26, 27: 27
}

In [ ]:
reverse_region_mappings = {v: k for k, v in region_mappings.items()}

In [ ]:
def load_and_process_data(folder):
    all_files = glob.glob(folder + "/*.csv")
    data_frames = []
    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']

    for file in all_files:
        region_code = int(file.split('_')[1])
        df = pd.read_csv(file, header=1, names=columns)
        df.drop(columns=['empty'], inplace=True)
        df = df[df['VHI'] != -1]
        df.insert(0, 'region_id', region_code)
        df['Year'] = df['Year'].str.extract(r'(\d+)', expand=False)
        df.dropna(subset=['Year'], inplace=True)  # Видаляємо рядки з NaN у стовпці Year
        df['Year'] = df['Year'].astype(int)
        data_frames.append(df)

    combined_df = pd.concat(data_frames).drop_duplicates().reset_index(drop=True)
    combined_df['region_id'] = combined_df['region_id'].map(region_mappings)
    return combined_df


In [ ]:
data = load_and_process_data('data')
print(data, '\n')

In [ ]:
def analyze_region_vhi(df, region_code, year_range):
    df_filtered = df[(df['region_id'] == region_code) & (df['Year'].between(year_range[0], year_range[1]))]
    if not df_filtered.empty:
        min_vhi = df_filtered['VHI'].min()
        max_vhi = df_filtered['VHI'].max()
        original_region_code = reverse_region_mappings.get(region_code, None)
        region_name = region_names_ukr.get(original_region_code, "Невідомий регіон")
        print(f"{Fore.BLUE}[+] {region_name}, {Fore.RED}min: {min_vhi}, {Fore.GREEN}max: {max_vhi}")


In [ ]:
for reg in range(1, 26):
    analyze_region_vhi(data, reg, (1985, 2010))

In [ ]:
def analyze_drought_years(df, year_range=("1981", "2024"), regions=None, extreme_threshold=15, moderate_threshold=(15, 35), affected_threshold=20):
    extreme_years = []
    moderate_years = []
    if regions:
        df = df[df['region_id'].isin(regions)]

    unique_regions = df['region_id'].nunique()
    min_affected = (affected_threshold / 100) * unique_regions

    for year in range(int(year_range[0]), int(year_range[1]) + 1):
        year_data = df[df['Year'] == year]
        if year_data.empty:
            continue

        extreme_count = (year_data['VHI'] < extreme_threshold).sum()
        moderate_count = year_data['VHI'].between(moderate_threshold[0], moderate_threshold[1]).sum()

        if extreme_count >= min_affected:
            extreme_years.append(year)
        if moderate_count >= min_affected:
            moderate_years.append(year)

    return extreme_years, moderate_years
extreme_drought_years, moderate_drought_years = analyze_drought_years(data, year_range=("1999", "2001"), regions=[3, 5, 12])
print("Роки екстремальних посух:", extreme_drought_years)
print("Роки помірних посух:", moderate_drought_years)